# Guía 2: SQL (Solución)

En esta guía usted deberá realizar consultas SQL en un servidor virtual con PostgreSQL que contiene
datos de películas extraídos de IMDb. El esquema de los datos es el siguiente:

* $\color{green}{\textbf{pelicula}}(\color{blue}{\underline{\text{nombre}}}, \color{blue}{\underline{\text{anho}}}, \color{blue}{\text{calificacion}})$ 
* $\color{green}{\textbf{actor}}(\color{blue}{\underline{\text{nombre}}}, \color{blue}{\text{genero}})$
* $\color{green}{\textbf{personaje}}(\color{blue}{\underline{\text{p_nombre}}}, \color{blue}{\underline{\text{p_anho}}},  \color{blue}{\underline{\text{a_nombre}}}, \color{blue}{\underline{\text{personaje}}})$

La tabla $\color{green}{\textbf{personaje}}$ usa llaves foráneas que hacen referencia a las tablas de $\color{green}{\textbf{actor}}(\color{blue}{\underline{\text{a_nombre}}})$ y $\color{green}{\textbf{pelicula}}(\color{blue}{\underline{\text{p_nombre}}}, \color{blue}{\underline{\text{p_anho}}})$.

Para iniciar el servidor virtual, instalar la base de datos postgres, y descargar los datos e importarlos, debe correr el siguiente bloque:



In [1]:
# install
!apt update
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
# set connection
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres
%sql create schema guia;
!wget -cO - https://github.com/IIC2413/Syllabus-2022-2/raw/main/Gu%C3%ADas/extra/guia2.sql > /content/guia2.sql
!psql postgres < /content/guia2.sql
%sql set search_path to guia;

from google.colab import output
output.clear()

Ejecute la siguiente consulta para probar que todo ande bien:

In [2]:
%sql select * from pelicula;

 * postgresql+psycopg2://@/postgres


,nombre,anho,calificacion
0,The Shawshank Redemption,1994,9.2
1,The Godfather,1972,9.2
2,The Godfather: Part II,1974,9.0
3,The Dark Knight,2008,8.9
4,12 Angry Men,1957,8.9
...,...,...,...
245,The King's Speech,2010,8.0
246,The Avengers,2012,8.0
247,Lagaan: Once Upon a Time in India,2001,8.0
248,Beauty and the Beast,1991,8.0


Para ejecutar consultas multi-lineas use el tag %%sql:

In [3]:
%%sql
SELECT *
FROM pelicula
where nombre ilike '%terminator%';

 * postgresql+psycopg2://@/postgres


,nombre,anho,calificacion
0,Terminator 2: Judgment Day,1991,8.5
1,The Terminator,1984,8.0


Ahora, debe diseñar las consultas que resuelvan las siguientes preguntas usando los operadores vistos en clases.

# SQL
## Pregunta 1
Las películas de los 80s, ordenadas por calificación de mayor a menor.

In [4]:
%%sql
SELECT *
FROM pelicula
WHERE anho BETWEEN 1980 AND 1989
ORDER BY calificacion DESC;

 * postgresql+psycopg2://@/postgres


,nombre,anho,calificacion
0,Star Wars: Episode V - The Empire Strikes Back,1980,8.7
1,Raiders of the Lost Ark,1981,8.5
2,Back to the Future,1985,8.5
3,Nuovo Cinema Paradiso,1988,8.4
4,The Shining,1980,8.4
5,Hotaru no haka,1988,8.4
6,Aliens,1986,8.4
7,Once Upon a Time in America,1984,8.4
8,Das Boot,1981,8.3
9,Star Wars: Episode VI - Return of the Jedi,1983,8.3


## Pregunta 2
Los nombres de los personajes que ha interpretado su actriz/actor favorito en los datos, ordenados por año.

In [5]:
%%sql
SELECT personaje
FROM personaje
WHERE a_nombre = 'Palin, Michael'
ORDER BY p_anho;

 * postgresql+psycopg2://@/postgres


,personaje
0,First Swallow-Savvy Guard
1,Dennis
2,Peasant 2
3,Right Head
4,Sir Galahad the Pure
5,Narrator
6,King of Swamp Castle
7,Brother Maynard's Brother
8,Leader of The Knights Who Say NI!
9,Eddie


# Pregunta 3
Las películas en las que participó su actriz/actor favorito en los datos, ordenadas por calificación de mayor a menor.

In [6]:
%%sql
SELECT nombre
FROM personaje, pelicula
WHERE nombre = p_nombre AND anho = p_anho
    AND a_nombre like 'McGregor, Ewan'
ORDER BY calificacion;

 * postgresql+psycopg2://@/postgres


,nombre
0,Star Wars: Episode VII - The Force Awakens
1,Trainspotting


## Pregunta 4
Los nombres de los personajes interpretados por mujeres, en películas de los 90s, con calificación mayor a 8,7.

In [7]:
%%sql
SELECT personaje
FROM personaje, pelicula, actor
WHERE pelicula.anho = personaje.p_anho
    AND pelicula.nombre = personaje.p_nombre
    AND actor.nombre = personaje.a_nombre
    AND anho BETWEEN 1990AND 1999
    AND calificacion > 8.7;

 * postgresql+psycopg2://@/postgres


,personaje
0,Winston Wolfe's Girlfriend At Party
1,Fourth Man
2,Jody
3,Herself - Schindler Mourner
4,Ghetto Girl
...,...
355,Henry Rosner
356,Support Group Member
357,Guard
358,Moresby Batter


## Pregunta 5
Las películas de la saga "The Lord of the Rings" (usando el prefijo de su nombre) ordenadas por calificación y por año.

In [8]:
%%sql
SELECT nombre
FROM pelicula
WHERE nombre like 'The Lord of the Rings%'
ORDER BY calificacion, anho;

 * postgresql+psycopg2://@/postgres


,nombre
0,The Lord of the Rings: The Two Towers
1,The Lord of the Rings: The Fellowship of the Ring
2,The Lord of the Rings: The Return of the King


## Pregunta 6
Los nombres de los actores que interpretan más de un personaje en la misma película.

In [9]:
%%sql
SELECT DISTINCT personaje1.a_nombre
FROM personaje AS personaje1, personaje AS personaje2
WHERE personaje1.p_nombre = personaje2.p_nombre
    AND personaje1.p_anho = personaje2.p_anho
    AND personaje1.a_nombre = personaje2.a_nombre
    AND personaje1.personaje <> personaje2.personaje;

 * postgresql+psycopg2://@/postgres


,a_nombre
0,"Abbott, Frankie"
1,"Abbou, Bernard"
2,"Adán, Pablo"
3,"Alazraqui, Carlos"
4,"Albertson, Jeff"
...,...
321,"Wisher Jr., William"
322,"Worsham, John"
323,"Yee, Mitchell (I)"
324,"Yoshida, Rihoko"


## Pregunta 7
Las películas en que actúan juntos Uma Thurman y Samuel L. Jackson.

In [10]:
%%sql
SELECT personaje1.p_nombre
FROM personaje AS personaje1, personaje AS personaje2
WHERE personaje1.p_nombre = personaje2.p_nombre
    AND personaje1.p_anho = personaje2.p_anho
    AND personaje1.a_nombre = 'Thurman, Uma'
    AND personaje2.a_nombre = 'Jackson, Samuel L.'

 * postgresql+psycopg2://@/postgres


,p_nombre
0,Pulp Fiction


In [11]:
 %%sql
SELECT p_nombre, p_anho FROM personaje
WHERE a_nombre='Thurman, Uma'
INTERSECT
SELECT p_nombre, p_anho FROM personaje
WHERE a_nombre='Jackson, Samuel L.';

 * postgresql+psycopg2://@/postgres


,p_nombre,p_anho
0,Pulp Fiction,1994


## Pregunta 8
Las películas en que actúa Uma Thurman y **no** Samuel L. Jackson.

In [12]:
%%sql
SELECT p_nombre, p_anho FROM personaje
WHERE a_nombre='Thurman, Uma'
EXCEPT
SELECT p_nombre, p_anho FROM personaje
WHERE a_nombre='Jackson, Samuel L.';

 * postgresql+psycopg2://@/postgres


,p_nombre,p_anho
0,Kill Bill: Vol. 1,2003
1,Kaze no tani no Naushika,1984


## Pregunta 9
Los pares de actores que aparecen juntos en más de una película. Cada par debe aparecer una sola vez, es decir, si $(A,B)$ aparece, no debe aparecer $(B,A)$, pues es el mismo par. Tampoco se deben incluir pares de la forma $(A,A)$.

In [13]:
%%sql
SELECT DISTINCT personaje1.a_nombre, personaje2.a_nombre
FROM personaje AS personaje1, personaje AS personaje2,
    personaje AS personaje3, personaje AS personaje4
WHERE personaje1.p_nombre = personaje2.p_nombre
  AND personaje1.p_anho = personaje2.p_anho
  AND personaje1.a_nombre < personaje2.a_nombre
  AND personaje3.p_nombre = personaje4.p_nombre
  AND personaje3.p_anho = personaje4.p_anho
  AND personaje1.a_nombre = personaje3.a_nombre
  AND personaje2.a_nombre = personaje4.a_nombre
  AND personaje1.p_nombre <> personaje3.p_nombre
  AND personaje1.p_anho <> personaje3.p_anho;

 * postgresql+psycopg2://@/postgres


,a_nombre,a_nombre
0,"Abbink, Marc","McCabe, Orion"
1,"Acevedo, Gino (I)","Appleby, Noel"
2,"Acevedo, Gino (I)","Astin, Sean"
3,"Acevedo, Gino (I)","Baker, Sala"
4,"Acevedo, Gino (I)","Bean, Sean"
...,...,...
5371,"Weaving, Hugo","Wood, Elijah (I)"
5372,"Wenham, David","Wood, Elijah (I)"
5373,"White, Dan (I)","Wilkerson, Guy"
5374,"Williams, Robert (I)","Wood, Wilson"


## Pregunta 10
La película con calificación más alta.

In [14]:
%%sql 
SELECT nombre, anho
FROM pelicula
EXCEPT
SELECT pelicula1.nombre, pelicula1.anho
FROM pelicula AS pelicula1, pelicula AS pelicula2
WHERE pelicula1.calificacion < pelicula2.calificacion;

 * postgresql+psycopg2://@/postgres


,nombre,anho
0,The Shawshank Redemption,1994
1,The Godfather,1972
